<a href="https://colab.research.google.com/github/Rivero-Romero/Deep_Learning/blob/main/01_Exploracion_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install kaggle library
!pip install kaggle

In [ ]:
# Upload kaggle credential
from google.colab import files

uploaded = files.upload()


In [ ]:
# Prepare kaggle environment
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Download dataset
!kaggle datasets download -d kaustubhb999/tomatoleaf

In [ ]:
!unzip tomatoleaf.zip

In [ ]:
!ls /content/

In [ ]:
!unzip /content/tomatoleaf.zip -d /content/tomatoleaf

In [ ]:
!ls /content/tomatoleaf

In [ ]:
import os
import pandas as pd

base_path = "/content/tomatoleaf/tomato/train"

# Getting categories from folder
categories = [f"{base_path}/{folder}" for folder in os.listdir(base_path) if os.path.isdir(f"{base_path}/{folder}")]

# Getting raw data from folder path
rows = []

for category in categories:
  images_path = [f"{category}/{file}" for file in os.listdir(category)]

  for image_path in images_path:
    rows.append([category.split("/")[-1], image_path])

columns = ["CATEGORY", "IMAGE_PATH"]

df = pd.DataFrame(rows, columns=columns)
df


In [ ]:
# Getting basic statistic

import cv2
import numpy as np

images_path = df["IMAGE_PATH"].values.tolist()
medians_images = []

for image_path in images_path:
  gray_image = cv2.imread(image_path, 0)
  resized_image = cv2.resize(gray_image, (256, 256))
  medians_images.append([image_path, np.median(resized_image)])

df_medians = pd.DataFrame(medians_images, columns=["IMAGE_PATH", "MEDIAN"])
df_medians

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
plt.hist(df_medians["MEDIAN"], bins=50, color='skyblue', edgecolor='black')
plt.title("Distribución de las Median de las Imágenes")
plt.xlabel("Mediana de los valores de píxeles")
plt.ylabel("Frecuencia")
plt.show()

In [ ]:
import seaborn as sns

plt.figure(figsize=(8,6))
sns.boxplot(data=df_medians["MEDIAN"], color='skyblue')
plt.title("Boxplot de las Median de las Imágenes")
plt.xlabel("Mediana de los valores de píxeles")
plt.show()

In [ ]:
# Cálculo de los cuartiles y el IQR
Q1 = df_medians['MEDIAN'].quantile(0.25)
Q3 = df_medians['MEDIAN'].quantile(0.75)
IQR = Q3 - Q1

# Definir los umbrales para detectar outliers
low_limit = Q1 - 1.5 * IQR
high_limit = Q3 + 1.5 * IQR

print(f"Quartil 1: {Q1}")
print(f"Quartil 3: {Q3}")
print(f"IQR: {IQR}")
print(f"Criteria filtering low: {low_limit}")
print(f"Criteria filtering high: {high_limit}")

# Detectar outliers
outliers_df = df_medians[ (df_medians['MEDIAN'] < low_limit) | (df_medians['MEDIAN'] > high_limit) ]

# Ver los outliers
print("Outliers detectados:")
print(outliers_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Suponemos que ya tienes el dataframe 'df_medians' con las medianas
# Usaremos 'df_medians' para filtrar outliers

# Definimos el criterio para detectar outliers
CRITERIA = 2

# Calcular el rango intercuartílico (IQR)
Q1 = df_medians['MEDIAN'].quantile(0.25)
Q3 = df_medians['MEDIAN'].quantile(0.75)
IQR = Q3 - Q1

# Filtrar las imágenes que están fuera del rango intercuartílico
filter = (df_medians['MEDIAN'] < Q1 - CRITERIA * IQR) | (df_medians['MEDIAN'] > Q3 + CRITERIA * IQR)
filtered_df = df_medians.loc[filter, ["IMAGE_PATH"]]

# Imprimir el número de outliers encontrados
print(f"There are {len(filtered_df)} outliers")

# Tomamos una muestra aleatoria de 16 imágenes outliers para visualizarlas
sample_images = filtered_df.sample(16, random_state=42)["IMAGE_PATH"].values.tolist()

# Crear una figura y subgráficas (4x4)
fig, axs = plt.subplots(4, 4, figsize=(12, 12))

# Iterar sobre las imágenes y mostrarlas en las subgráficas
for i, ax in enumerate(axs.flat):
    image = cv2.imread(sample_images[i])
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    ax.axis('off')  # Desactivar los ejes para mostrar solo la imagen

# Mostrar las imágenes
plt.tight_layout()
plt.show()


In [ ]:
# Filtrar las imágenes que no son outliers
df_filter = pd.merge(df, filtered_df, on='IMAGE_PATH', how='left', indicator=True)

# Filtrar solo las filas que están solo en 'df' y no en 'filtered_df'
df_filter = df_filter[df_filter['_merge'] == 'left_only']

# Mostrar el dataframe resultante
df_filter

In [ ]:
import matplotlib.pyplot as plt
# Contar las imágenes por categoría
value_counts = df_filter['CATEGORY'].value_counts()
# Crear el gráfico de barras
plt.figure(figsize=(10, 6))  # Ajustar el tamaño del gráfico si es necesario
plt.bar(value_counts.index, value_counts.values, color='skyblue')
# Personalizar el título y las etiquetas
plt.title('Cantidad de Imágenes por Categoría', fontsize=16)
plt.xlabel('Categoría', fontsize=14)
plt.ylabel('Cantidad de Imágenes', fontsize=14)
# Rotar las etiquetas del eje X para mejorar la legibilidad si son largas
plt.xticks(rotation=45, ha='right', fontsize=12)
# Agregar una cuadrícula para facilitar la lectura
plt.grid(axis='y', linestyle='--', alpha=0.7)
# Mostrar el gráfico
plt.tight_layout()  # Asegura que los elementos no se sobrepongan
plt.show()

In [ ]:
# Función para clasificar la salud de la planta
def check_health(string):
    if "healthy" in string:  # Verifica si la palabra "healthy" está en la categoría
        return "healthy"
    else:
        return "no healthy"

# Crear una nueva columna 'CATEGORY_BIN' en df o df_filter
df_transformed = df.copy()  # Usar df o df_filter, dependiendo del que tengas
df_transformed["CATEGORY_BIN"] = df_transformed["CATEGORY"].apply(lambda value: check_health(value.lower()))

# Ver los primeros registros para verificar la nueva columna
print(df_transformed.head())

In [ ]:
import plotly.express as px

# Contar las imágenes en cada categoría binaria
category_counts = df_transformed.groupby("CATEGORY_BIN").size().reset_index(name="COUNT")

# Crear el gráfico de barras
fig = px.bar(category_counts,
             x="CATEGORY_BIN",
             y="COUNT",
             title="Distribution of Images by Health Status",  # Título del gráfico
             labels={"CATEGORY_BIN": "Health Status", "COUNT": "Number of Images"},  # Etiquetas de los ejes
             color="CATEGORY_BIN",  # Usar color para diferenciar las categorías
             color_discrete_map={"healthy": "green", "no healthy": "red"})  # Colores personalizados para las categorías

# Mostrar el gráfico
fig.show()


In [ ]:
# Saving dataframe as csv
df_transformed.to_csv("transformed_categories.csv", index=False)